In [174]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import pandas as pd

In [175]:
#bring in df
df = pd.read_csv('database.csv')
df.head()

,text,date,user.username
0,OCU buys Scottish civils contractor RJ McLeod ...,"Jun 12, 2024 · 10:02 AM UTC",@ConstructionEnq
1,Construction output falls for third month runn...,"Jun 12, 2024 · 8:51 AM UTC",@ConstructionEnq
2,Balfour wins design for £690m Skye transmissio...,"Jun 12, 2024 · 7:19 AM UTC",@ConstructionEnq
3,Construction output falls for third month runn...,"Jun 12, 2024 · 7:19 AM UTC",@ConstructionEnq
4,Plan in for Leeds 27-storey block of flats dlv...,"Jun 12, 2024 · 6:33 AM UTC",@ConstructionEnq


In [176]:
#bring in model, tokeniser and initialise labels
roberta = "cardiffnlp/twitter-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

c:\Users\edwar\Desktop\Hackathons\Sheffield hackthon 11-06-2024\sheffield_hack\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [177]:
def tweet_encoder(tweet, m_tokenizer=tokenizer):
   """Tweet tokenizer and model for 1 item"""
   encoded_tweet = m_tokenizer(tweet, return_tensors='pt')
   output = model(**encoded_tweet)
   score = output[0][0].detach().numpy()
   return score

In [178]:
def results(Neg, Neu, Pos, Score):
    """Update labels with score"""
    Neg += Score[0]
    Neu += Score[1]
    Pos += Score[2]
    return Neg, Neu, Pos

In [179]:
if __name__ == '__main__':
   global df
   Negative = 0
   Neutral = 0
   Positive = 0
   sentiment_labels = [Negative, Neutral, Positive]
   #this is the part where we loop through each row in the series
   df['Negative'] =  None
   df['Neutral'] = None
   df['Positive'] = None
   
   for index, row in df.iterrows():
      tweet = row['text'] #each tweet text is selected
      output = tweet_encoder(row['text']) #tweet is tokenised, and scored
      sentiment = softmax(output) #output is normalised
      df.loc[index, "Negative"] = sentiment[0] #inserts negative score
      df.loc[index, "Neutral"] = sentiment[1] #inserts neutral score
      df.loc[index, "Positive"] = sentiment[2] #inserts positive score

In [180]:
df.head()

,text,date,user.username,Negative,Neutral,Positive
0,OCU buys Scottish civils contractor RJ McLeod ...,"Jun 12, 2024 · 10:02 AM UTC",@ConstructionEnq,0.040948,0.922272,0.03678
1,Construction output falls for third month runn...,"Jun 12, 2024 · 8:51 AM UTC",@ConstructionEnq,0.212693,0.746651,0.040656
2,Balfour wins design for £690m Skye transmissio...,"Jun 12, 2024 · 7:19 AM UTC",@ConstructionEnq,0.009108,0.670077,0.320815
3,Construction output falls for third month runn...,"Jun 12, 2024 · 7:19 AM UTC",@ConstructionEnq,0.568673,0.412028,0.019299
4,Plan in for Leeds 27-storey block of flats dlv...,"Jun 12, 2024 · 6:33 AM UTC",@ConstructionEnq,0.031066,0.934748,0.034186


In [194]:
fin_neg = df['Negative'].sum()*-1
fin_neu = df['Neutral'].sum()
fin_pos = df['Positive'].sum()

score = (fin_neg +(fin_neu*0) +fin_pos)/len(df.index)